# Convert Data Files to WISP Format for Spectral Fitting

**Author:** Mitchell Revalski

**Adapted:** Kalina V. Nedkova and James Colbert

**Last Updated:** July 12, 2024

***

## Jupyter Notebook for the MUSE Ultra Deep Field Program

This notebook was created by Dr. Mitchell Revalski to convert the HST passage and VLT MUSE spectroscopic files provided by Dr. Nor Pirzkal and Dr. Matteo Fossati into a format that is compatible with the version of the WISP spectroscopic fitting routine that is being used for the MUDF program.

Please send questions, comments, and suggestions to Mitchell Revalski at [mrevalski@stsci.edu](mrevalski@stsci.edu). Thank you, and have a nice day!

***

## This version has been adapted for PASSAGE

The updated version of this notebook converts JWST 1D spectroscopic files produced by grizli into a format that is compatible with the version of the WISP spectroscopic fitting routine that is being used for the PASSAGE program.

Moreover, in an optional step 3, 1D spectroscopic files are created for the individual orientations (R grism & C grism). The location of the beam files from grizli needs to be specified for this step. 

Please send questions, comments, and suggestions w.r.t. the PASSAGE updates to Kalina at [knedkova@stsci.edu](knedkova@stsci.edu). 
Special thank you to James Colbert for fixing a bug that caused this code to crash if the F150W grism filter does not exist. 

***


## Software

The Python environment (ac6) used to run this notebook was built using conda:<br>
<code>
(base) asteropaios:Documents mrevalski$ conda create -n ac6 stsci
(base) asteropaios:Documents mrevalski$ conda activate ac6
(ac6) asteropaios:Documents mrevalski$ conda update drizzlepac
(ac6) asteropaios:Documents mrevalski$ conda deactivate
</code>

The jupyter notebook can then be run using the following call sequence:<br>
<code>
(base) asteropaios:conversion mrevalski$ conda activate ac6
(ac6) asteropaios:conversion mrevalski$ jupyter notebook
</code>

The critical package versions include:<br>
<code>
Python Version [GCC Clang 10.0.0] 3.6.13 |Anaconda, Inc.|
(default, Feb 23 2021, 12:58:59) 
numpy Version -> 1.19.2 
astropy Version -> 4.0.2
</code>  

***

## Table of Contents <a class="anchor" id="tag0"><a>


**&emsp; 1) [Setup Options](#setup)<br>**
**&emsp; 2) [Convert PASSAGE](#passage)<br>**
**&emsp; 3) [Create 1D spectra for each orientation](#orientations)<br>**
    
*** 

## Imports

The following packages are required to run the Jupyter Notebook:
 - *os* - change and make directories
 - *glob* - determine files in folders
 - *numpy* - math and array functions
 - *pandas* - data structure functions    
 - *astropy* - file and data handling
      - *io.fits* - import FITS files
      - *table.Table* - display formatted tables

In [1]:
import os
import glob
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table, vstack
from time import time

import grizli
print(grizli.__version__)

from grizli import utils, multifit
from grizli.pipeline import auto_script

utils.set_warnings()

1.11.10
Failed to `import dust_attenuation`
Install from the repo with $ pip install git+https://github.com/karllark/dust_attenuation.git


## Setup Options  <a class="anchor" id="setup"><a>
[Table of Contents](#tag0)

In [2]:
# main directory
main_directory = '/Users/knedkova/Work/2024PASSAGE/'
par_names = ['Par28']

# grism data directory
spec1D_directory = main_directory+'data/'+par_names[0]+'/spec1D/'

# where all the parallel field data live
data_directory = main_directory+'data/'+par_names[0]+'/'

In [3]:
os.chdir(spec1D_directory)

## Convert passage  <a class="anchor" id="passage"><a>
[Table of Contents](#tag0)

1. In order for the wavelet solutions to work, the 'RADESYS' keyword in the header must be either updated or removed. 

In [4]:
#check if already updated for first object. If yes -> do nothing, if not -> update RADESYS in all fits headers

for k in par_names:
    files = glob.glob(main_directory+'data/'+str(k)+'/Stamps/*.fits')
    test_file = fits.open(files[0])
    header = test_file[2].header
    
    if header['RADESYS'] != 'ICRS':
        print('Headers are not updated. Updating all now.')
        hdulist = fits.open(main_directory+'data/'+str(k)+'/Stamps/*.fits')

        for j in files:
            # Open the file header for viewing and load the header
            hdulist = fits.open(j)
            for i in range(len(hdulist)):
                header = hdulist[i].header
                try: header['RADESYS'] = 'ICRS'
                except: print('no RADESYS in header')
            
            hdulist.writeto(j, overwrite='True')


2. Now create 1D spectral files

In [12]:
# flag for creating the output files regardless of whehter they already exist
# True = create new files
# False = create new files only if they do not exist 
create_files_anyway = True

In [14]:
for i in par_names:
    
    try:os.mkdir(data_directory)
    except:print('I did not make your directory')
    try:
        os.chdir(data_directory)
        os.mkdir('Spectra')
    except:print('I did not make your directory')
    
    os.chdir(spec1D_directory+str(i))
    print('moving to:', spec1D_directory+str(i))
    files = sorted(glob.glob('*1D.fits'))

    # Check if converted files already exist. If they do not, carry on with the conversion. 
    # Otherwise, this step can be skipped. 
    # Here, I just check if there are more converted files than objects (there can be 1-3 per object)
    # depending on how many filters are available for each object
    already_converted_files = glob.glob(data_directory + 'Spectra/*.dat')
    if len(files) >= len(already_converted_files) or create_files_anyway == True:
        print(len(already_converted_files))
        print(len(files))
    
        print('\nThere are ' + str(len(files)) + ' PASSAGE files to convert.\n')
        for f in files:
            os.chdir(spec1D_directory+str(i))
            #os.chdir(spec1D_directory+par_names[0]+'/')
            fff = fits.open(f)
            print(fff.info)

            for ext in range(1, len(fff)):
                
                tb = Table.read(fff[ext], format='fits').to_pandas()
                t_out = pd.DataFrame({})
            
                t_out['wave'] = tb['wave']
                t_out['flux'] = tb['flux']/tb['flat']
                t_out['error'] = tb['err']/tb['flat']
                t_out['contam'] = tb['contam']/tb['flat']
                t_out['zeroth'] = np.zeros(len(tb['wave'])).astype('int')
                t_out = Table.from_pandas(t_out)
                t_out = t_out.filled(0.0) # Replace nans with zeros
                # Spectra dispersed beyond the chip have zero fluxes that must be replaced to prevent crashes in fitting.
                t_out['flux'][np.where(t_out['flux'] == 0.0)] = np.median(t_out['flux'][np.where(t_out['flux'] != 0.0)])
                t_out['error'][np.where(t_out['error'] == 0.0)]=np.median(t_out['error'][np.where(t_out['error'] != 0.0)])
    
                print('saving to '+ data_directory + 'Spectra' )
                os.chdir(data_directory + 'Spectra' )
    
                # Write out the updated files.
                if fff[ext].header['EXTNAME'] == 'F200W':
                    _indexes_ = [i for i in range(len(fff[ext].data))]
                    try:_indexes_ = [i for i in _indexes_ if t_out['wave'][i] > max(Table.read(fff[ext-1])['wave']) ]
                    except:pass
                    t_out[_indexes_].write(os.path.basename(f).replace('1D.fits', 'G200_1D.dat'), 
                            format='ascii.fixed_width_two_line', overwrite=True)      
                
                # Write out the updated files.
                if fff[ext].header['EXTNAME'] == 'F150W':
                    #t_out.write(os.path.basename(f).replace('1D.fits', 'G150_1D_uncomb.dat'), 
                    t_out.write(os.path.basename(f).replace('1D.fits', 'G150_1D.dat'), 
                            format='ascii.fixed_width_two_line', overwrite=True)
                    
                elif fff[ext].header['EXTNAME'] == 'F115W':
                    t_out.write(os.path.basename(f).replace('1D.fits', 'G115_1D.dat'), 
                            format='ascii.fixed_width_two_line', overwrite=True)

        # THE BELOW IS NO LONGER NEEDED. 
    
        # Now that the .dat files have been created/updated, need to combine the F200W with the F150W
        # # since the next code expects only 2 grism filters.
        # os.chdir(data_directory + '/Spectra' )

        # target_dir = data_directory + 'Spectra'
        # F200s = glob.glob(target_dir+'/*G200_1D.dat')
        
        # for i in range(len(F200s)):
            # F150s = F200s[i].split('G200_1D.dat')[0]+str('G150_1D_uncomb.dat')
            # tb_200 = Table.read(F200s[i], format='ascii')
            # if os.path.exists(F150s):
                # tb_150 = Table.read(F150s, format='ascii')
                    
            # tout = vstack([tb_150, tb_200])
            # print(os.path.basename(F150s).replace('G150_1D_uncomb.dat', 'G150_1D.dat'))
                
            # tout.write(os.path.basename(F150s).replace('G150_1D_uncomb.dat', 'G150_1D.dat'), 
                            # format='ascii.fixed_width_two_line', overwrite=True)


    else: print('It appears the .dat files were already created for this field. Skipping this step.')
    


I did not make your directory
I did not make your directory
moving to: /Users/knedkova/Work/2024PASSAGE/data/Par28/spec1D/Par28
26875
2754

There are 2754 PASSAGE files to convert.

<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x29842b150>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298203290>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298200890>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x297cdf910>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x29840e610>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2983f0f10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2981df290>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2981defd0>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
savi

2024-07-30 15:49:49,596 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-30 15:49:49,597 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-30 15:49:49,597 - stpipe - WARNING - 


<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x298445510>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29724e890>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29840c450>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29840ab10>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x29840d990>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29827be90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2981de350>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29855d310>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.Prim

2024-07-30 15:49:57,630 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-30 15:49:57,630 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-30 15:49:57,631 - stpipe - WARNING - 


<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x28a2a2910>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298278e50>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298299b50>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2983ecf90>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x298432e90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29842b610>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298433710>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298431d90>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.Prim

...

<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x29829e910>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2982a6f50>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29804aad0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298431d90>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x29840cd90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x297cb8b90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2984cd810>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2984ce350>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.Prim

2024-07-30 15:57:06,813 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-30 15:57:06,813 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-30 15:57:06,814 - stpipe - WARNING - 


<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x298564b90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298337d10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x29840a9d0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x297c3af10>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x2944aca90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x297c97dd0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2984e4bd0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2984b4b10>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.Prim

2024-07-30 15:57:21,053 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-30 15:57:21,054 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-30 15:57:21,054 - stpipe - WARNING - 
2024-07-30 15:57:21,060 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-30 15:57:21,061 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-30 15:57:21,061 - stpipe - WARNING - 


<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x298105050>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x297cd3c90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298506e10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2981be110>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x29829e390>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2981b6750>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2981d9990>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x298317310>]>
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
saving to /Users/knedkova/Work/2024PASSAGE/data/Par28/Spectra
<bound method HDUList.info of [<astropy.io.fits.hdu.image.Prim

In [269]:
os.getcwd()

'/Users/knedkova/Work/PASSAGE/data_old/spec1D'

## Create 1D spectroscopic file for the R and C orientations  <a class="anchor" id="orientations"><a>
[Table of Contents](#tag0)

In [86]:
photcat_file = glob.glob(data_directory+'DATA/DIRECT_GRISM/*_photcat.fits')
photcat = Table.read(photcat_file[0])

speccat_file = glob.glob(data_directory+'DATA/DIRECT_GRISM/*_speccat.fits')
speccat =  Table.read(speccat_file[0])


In [7]:
def make_table(tb):
    t_out = pd.DataFrame({})
    t_out['wave'] = tb['wave']
    t_out['flux'] = tb['flux']/tb['flat']
    t_out['error'] = tb['err']/tb['flat']
    t_out['contam'] = tb['contam']/tb['flat']
    t_out['zeroth'] = np.zeros(len(tb['wave'])).astype('int')
    t_out = Table.from_pandas(t_out)
    t_out = t_out.filled(0.0) # Replace nans with zeros
    # Spectra dispersed beyond the chip have zero fluxes that must be replaced to prevent crashes in fitting.
    t_out['flux'][np.where(t_out['flux'] == 0.0)] = np.median(t_out['flux'][np.where(t_out['flux'] != 0.0)])
    t_out['error'][np.where(t_out['error'] == 0.0)]=np.median(t_out['error'][np.where(t_out['error'] != 0.0)])

    return t_out

In [87]:


for i in par_names:
    beam_files = glob.glob(data_directory + 'beams/*.beams.fits')
    os.chdir(data_directory + 'Spectra/')
    start = time()

    for j in beam_files:
        id = int(j.split('/')[-1].split('_')[1].split('.')[0])
        z = speccat['redshift'][[k for k in range(len(speccat)) if speccat['id'][k] == id]]

        mb = multifit.MultiBeam(j, fcontam=0.1, sys_err=0.02,
                        min_sens=0.05, MW_EBV=-1, group_name='', verbose=False)

        Cgrism_beams = [mb.beams[k] for k in range(len(mb.beams)) if  mb.beams[k].grism.filter == 'GR150C']
        Rgrism_beams = [mb.beams[k] for k in range(len(mb.beams)) if  mb.beams[k].grism.filter == 'GR150R']
        
        if len(Cgrism_beams) > 0:
            mb_C = multifit.MultiBeam(beams=Cgrism_beams, fcontam=0.1, sys_err=0.02, 
                        min_sens=0.05, MW_EBV=-1, group_name='')
            
            # this catches cases where spectrum contains only zeros (very rare)
            # The fit will crash in such cases
            try:
                tfitC = mb_C.template_at_z(z, fitter='bounded')
                keys_C = mb_C.oned_spectrum(tfit=tfitC, bin=1).keys()
            except: keys_C = mb_C.oned_spectrum(bin=1).keys()
            
            for c in keys_C:
                t_out = make_table(mb_C.oned_spectrum(tfit=tfitC, bin=1)[c])
                t_out.write(os.path.basename(j).replace('beams.fits', c+'_1D_C.dat'), 
                            format='ascii.fixed_width_two_line', overwrite=True)

        if len(Rgrism_beams) > 0:
            mb_R = multifit.MultiBeam(beams=Rgrism_beams, fcontam=0.1, sys_err=0.02, 
                        min_sens=0.05, MW_EBV=-1, group_name='')

            # this catches cases where spectrum contains only zeros (very rare)
            # The fit will crash in such cases
            try:
                tfitR = mb_R.template_at_z(z, fitter='bounded')
                keys_R = mb_R.oned_spectrum(tfit=tfitR, bin=1).keys()
            except: keys_R = mb_R.oned_spectrum(bin=1).keys()
                
            for r in keys_R:
                t_out = make_table(mb_R.oned_spectrum(tfit=tfitR, bin=1)[r])
                t_out.write(os.path.basename(j).replace('beams.fits', r+'_1D_R.dat'), 
                            format='ascii.fixed_width_two_line', overwrite=True)


        end = time()
        print(end - start)




 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
4.974880933761597
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
10.830098152160645
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1

2024-07-31 07:38:23,553 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-31 07:38:23,554 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-31 07:38:23,554 - stpipe - WARNING - 
2024-07-31 07:38:23,620 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-31 07:38:23,628 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-31 07:38:23,633 - stpipe - WARNING - 
2024-07-31 07:38:25,063 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-31 07:38:25,073 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-31 07:38:25,109 - stpipe - WARNING - 
2024-07-31 07:38:25,208 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-pa

139.6674680709839
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
146.28913593292236
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
152.48337292671204
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) b

2024-07-31 07:44:22,548 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-31 07:44:22,556 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-31 07:44:22,557 - stpipe - WARNING - 


496.9595251083374
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5


...

19640.024255990982
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
19644.82120704651
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
19651.442130804062
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) b

2024-07-31 13:04:39,611 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-31 13:04:39,645 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-31 13:04:39,689 - stpipe - WARNING - 


19715.412452936172
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
19721.314621925354
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
19726.930011034012
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) 

2024-07-31 13:05:25,843 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-31 13:05:25,872 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-31 13:05:25,903 - stpipe - WARNING - 
2024-07-31 13:05:27,377 - stpipe - WARNING - /opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3502: RuntimeWarning: Mean of empty slice.
2024-07-31 13:05:27,379 - stpipe - WARNING -   return mean(axis=axis, dtype=dtype, out=out, **kwargs)
2024-07-31 13:05:27,380 - stpipe - WARNING - 


19761.845721006393
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
19767.853768110275
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
19774.04087114334
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) by an additional x 1.5
 ! Scale 0th order (B) b

In [11]:
plt.plot(mb_R.oned_spectrum()['F115W']['wave'], mb_R.oned_spectrum()['F115W']['flux'])
plt.plot(mb_R.oned_spectrum()['F115W']['wave'], mb_R.oned_spectrum()['F115W']['contam'])

In [10]:
import matplotlib.pylab as plt

In [25]:
from astropy.io import ascii as asc

In [75]:

base_path = "/Users/knedkova/Work/2024PASSAGE/data/"+ "Par"+ str(28)+ "/Spectra/Par"+ str(28)+ "_" + str(6).zfill(5)

specnameg1_R = (base_path + ".F115W_1D_R.dat")
specnameg2_R = (base_path + ".F150W_1D_R.dat")
specnameg3_R = (base_path + ".F200W_1D_R.dat")


In [78]:
tab_blue_R = asc.read(specnameg2_R, names=["lambda","flux","ferror","contam","zero"])
tab_blue_R_contam = np.copy(tab_blue_R)
tab_blue_R_contam["flux"] = tab_blue_R["flux"] + tab_blue_R["contam"]  

In [79]:
tab_blue_R

<Table length=99>
      lambda                flux                 ferror         contam  zero
     float64              float64               float64        float64 int64
------------------ --------------------- --------------------- ------- -----
12819.201485620451 1.232067831315147e-21 5.016545169530969e-21     0.0     0
12857.662061275081 1.232067831315147e-21 5.016545169530969e-21     0.0     0
12896.238018570266 1.232067831315147e-21 5.016545169530969e-21     0.0     0
12934.929703650672 1.232067831315147e-21 5.016545169530969e-21     0.0     0
12973.737463699392 1.232067831315147e-21 5.016545169530969e-21     0.0     0
13012.661646941073 1.232067831315147e-21 5.016545169530969e-21     0.0     0
 13051.70260264503 1.232067831315147e-21 5.016545169530969e-21     0.0     0
13090.860681128388 1.232067831315147e-21 5.016545169530969e-21     0.0     0
13130.136233759222 1.232067831315147e-21 5.016545169530969e-21     0.0     0
               ...                   ...                   ...     ...   ...
16736.007200170876 1.232067831315147e-21 5.016545169530969e-21     0.0     0
 16786.21818416955 1.232067831315147e-21 5.016545169530969e-21     0.0     0
16836.579801007407 1.232067831315147e-21 5.016545169530969e-21     0.0     0
 16887.09250258264 1.232067831315147e-21 5.016545169530969e-21     0.0     0
16937.756742149115 1.232067831315147e-21 5.016545169530969e-21     0.0     0
16988.572974320465 1.232067831315147e-21 5.016545169530969e-21     0.0     0
17039.541655074165 1.232067831315147e-21 5.016545169530969e-21     0.0     0
17090.663241755617 1.232067831315147e-21 5.016545169530969e-21     0.0     0
 17141.93819308226 1.232067831315147e-21 5.016545169530969e-21     0.0     0
 17193.36696914769 1.232067831315147e-21 5.016545169530969e-21     0.0     0

In [61]:
tab_blue_R_contam["flux"] = tab_blue_R["flux"] + tab_blue_R["contam"]  

In [62]:
tab_blue_R_contam

<Table length=139>
      lambda         flux          ferror          contam  zero
     float64       float64        float64         float64 int64
------------------ ------- ---------------------- ------- -----
 16818.99609744206     0.0 5.2211980925903045e-21     0.0     0
16857.031248307125     0.0 5.2211980925903045e-21     0.0     0
16895.152408270704     0.0 5.2211980925903045e-21     0.0     0
 16933.35977182565     0.0 5.2211980925903045e-21     0.0     0
16971.653533904624     0.0 5.2211980925903045e-21     0.0     0
17010.033889881088     0.0 5.2211980925903045e-21     0.0     0
17048.501035570305     0.0 5.2211980925903045e-21     0.0     0
 17087.05516723034     0.0 5.2211980925903045e-21     0.0     0
 17125.69648156305     0.0 5.2211980925903045e-21     0.0     0
               ...     ...                    ...     ...   ...
  22508.7267661933     0.0 5.2211980925903045e-21     0.0     0
22559.628136712276     0.0 5.2211980925903045e-21     0.0     0
22610.644610783933     0.0 5.2211980925903045e-21     0.0     0
22661.776448692577     0.0 5.2211980925903045e-21     0.0     0
22713.023911311095     0.0 5.2211980925903045e-21     0.0     0
 22764.38726010229     0.0 5.2211980925903045e-21     0.0     0
 22815.86675712021     0.0 5.2211980925903045e-21     0.0     0
 22867.46266501148     0.0 5.2211980925903045e-21     0.0     0
22919.175247016665     0.0 5.2211980925903045e-21     0.0     0
22971.004766971586     0.0 5.2211980925903045e-21     0.0     0

In [63]:
tab_blue_R

<Table length=139>
      lambda         flux          ferror          contam  zero
     float64       float64        float64         float64 int64
------------------ ------- ---------------------- ------- -----
 16818.99609744206     0.0 5.2211980925903045e-21     0.0     0
16857.031248307125     0.0 5.2211980925903045e-21     0.0     0
16895.152408270704     0.0 5.2211980925903045e-21     0.0     0
 16933.35977182565     0.0 5.2211980925903045e-21     0.0     0
16971.653533904624     0.0 5.2211980925903045e-21     0.0     0
17010.033889881088     0.0 5.2211980925903045e-21     0.0     0
17048.501035570305     0.0 5.2211980925903045e-21     0.0     0
 17087.05516723034     0.0 5.2211980925903045e-21     0.0     0
 17125.69648156305     0.0 5.2211980925903045e-21     0.0     0
               ...     ...                    ...     ...   ...
  22508.7267661933     0.0 5.2211980925903045e-21     0.0     0
22559.628136712276     0.0 5.2211980925903045e-21     0.0     0
22610.644610783933     0.0 5.2211980925903045e-21     0.0     0
22661.776448692577     0.0 5.2211980925903045e-21     0.0     0
22713.023911311095     0.0 5.2211980925903045e-21     0.0     0
 22764.38726010229     0.0 5.2211980925903045e-21     0.0     0
 22815.86675712021     0.0 5.2211980925903045e-21     0.0     0
 22867.46266501148     0.0 5.2211980925903045e-21     0.0     0
22919.175247016665     0.0 5.2211980925903045e-21     0.0     0
22971.004766971586     0.0 5.2211980925903045e-21     0.0     0

In [84]:
plt.plot( tab_blue_R_contam['flux'] - tab_blue_R['flux'], 'k-')
plt.plot( tab_blue_R_contam['contam'], 'r-')
plt.show()

In [85]:
contam

NameError: name 'contam' is not defined